In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

alpha_key = os.getenv('ALPHA_VANTAGE_API_KEY')
news_key = os.getenv('NEWS_API_KEY')

print(f"Alpha Vantage Key: {alpha_key[:10]}..." if alpha_key else "Alpha Key missing!")
print(f"News API Key: {news_key[:10]}..." if news_key else "News Key missing!")

Alpha Vantage Key: YFKBRYHC4L...
News API Key: 5e13278f18...


In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_financial_news(company_name, days_back=7):
    """
    Fetch recent news articles about a company
    """
    NEWS_API_KEY = os.getenv('NEWS_API_KEY')
    
    # Calculate date range
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_back)
    
    # Format dates for API
    from_date = start_date.strftime('%Y-%m-%d')
    to_date = end_date.strftime('%Y-%m-%d')
    
    # Build API request
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q': company_name,
        'from': from_date,
        'to': to_date,
        'language': 'en',
        'sortBy': 'publishedAt',
        'apiKey': NEWS_API_KEY
    }
    
    # Make request
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        articles = response.json()['articles']
        
        # Convert to DataFrame
        news_data = []
        for article in articles:
            news_data.append({
                'title': article['title'],
                'description': article['description'],
                'published_at': article['publishedAt'],
                'source': article['source']['name'],
                'url': article['url']
            })
        
        df = pd.DataFrame(news_data)
        print(f"✅ Fetched {len(df)} articles about {company_name}")
        return df
    else:
        print(f"❌ Error: {response.status_code}")
        return None

In [3]:
# Test the news scraper
news_df = fetch_financial_news("Apple", days_back=3)

# Display the results
if news_df is not None:
    print("\n📰 Recent News Headlines:")
    print(news_df[['title', 'published_at', 'source']].head(10))

✅ Fetched 93 articles about Apple

📰 Recent News Headlines:
                                               title          published_at  \
0                              mlx-guided-grpo 2.1.1  2026-02-05T12:46:23Z   
1                      mlx-guided-grpo added to PyPI  2026-02-05T12:46:19Z   
2  Bad Bunny to discuss Super Bowl halftime perfo...  2026-02-05T12:39:42Z   
3  Get $600 Off This Lenovo ThinkBook With 16-Inc...  2026-02-05T12:36:27Z   
4  Kotak Mahindra Bank to add 500 engineers amid ...  2026-02-05T12:34:14Z   
5  Savannah Guthrie's missing mother 'still out t...  2026-02-05T12:34:13Z   
6  Official Apple Watch Series 11-compatible Solo...  2026-02-05T12:31:49Z   
7  Good Hoops Morning: To stay afloat, Ohio State...  2026-02-05T12:31:00Z   
8  Inside one of the ‘darkest days in the history...  2026-02-05T12:30:03Z   
9  Report: Galatasaray has €15 million purchase o...  2026-02-05T12:30:00Z   

                          source  
0                       Pypi.org  
1          

In [4]:
# Install required libraries for sentiment analysis
!pip install transformers torch

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load FinBERT model (specialized for financial sentiment)
print("Loading FinBERT model...")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
print("✅ FinBERT loaded successfully!")

def analyze_sentiment(text):
    """
    Analyze sentiment of financial text
    Returns: sentiment (positive/negative/neutral) and confidence score
    """
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    
    # Get prediction
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Get sentiment label and score
    sentiment_score = predictions.detach().numpy()[0]
    sentiment_labels = ['positive', 'negative', 'neutral']
    sentiment = sentiment_labels[sentiment_score.argmax()]
    confidence = sentiment_score.max()
    
    return sentiment, confidence

# Test it
test_text = "Apple reports record-breaking quarterly revenue, stock surges 10%"
sentiment, confidence = analyze_sentiment(test_text)
print(f"\nTest: '{test_text}'")
print(f"Sentiment: {sentiment} (confidence: {confidence:.2%})")

Loading FinBERT model...


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: ProsusAI/finbert
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ FinBERT loaded successfully!

Test: 'Apple reports record-breaking quarterly revenue, stock surges 10%'
Sentiment: positive (confidence: 88.66%)


In [6]:
print("Starting download...")

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

print("Step 1: Downloading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
print("✅ Tokenizer loaded!")

print("Step 2: Downloading model (this may take 2-5 minutes)...")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
print("✅ Model loaded!")

def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_score = predictions.detach().numpy()[0]
    sentiment_labels = ['positive', 'negative', 'neutral']
    sentiment = sentiment_labels[sentiment_score.argmax()]
    confidence = sentiment_score.max()
    return sentiment, confidence

print("Testing sentiment analysis...")
test_text = "Apple reports record-breaking quarterly revenue, stock surges 10%"
sentiment, confidence = analyze_sentiment(test_text)
print(f"\nTest: '{test_text}'")
print(f"Sentiment: {sentiment} (confidence: {confidence:.2%})")
print("\n🎉 Everything working!")

Starting download...
Step 1: Downloading tokenizer...
✅ Tokenizer loaded!
Step 2: Downloading model (this may take 2-5 minutes)...


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: ProsusAI/finbert
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Model loaded!
Testing sentiment analysis...

Test: 'Apple reports record-breaking quarterly revenue, stock surges 10%'
Sentiment: positive (confidence: 88.66%)

🎉 Everything working!


In [7]:
def get_news_with_sentiment(company_name, days_back=7):
    """
    Fetch news and analyze sentiment for each article
    """
    print(f"📰 Fetching news for {company_name}...")
    news_df = fetch_financial_news(company_name, days_back)
    
    if news_df is None or len(news_df) == 0:
        print("No news found!")
        return None
    
    print(f"🤖 Analyzing sentiment for {len(news_df)} articles...")
    
    # Analyze sentiment for each article
    sentiments = []
    confidences = []
    
    for idx, row in news_df.iterrows():
        # Combine title and description for better analysis
        text = f"{row['title']}. {row['description']}"
        
        try:
            sentiment, confidence = analyze_sentiment(text)
            sentiments.append(sentiment)
            confidences.append(confidence)
        except:
            sentiments.append('neutral')
            confidences.append(0.0)
    
    # Add to dataframe
    news_df['sentiment'] = sentiments
    news_df['confidence'] = confidences
    
    # Calculate overall sentiment score
    sentiment_counts = news_df['sentiment'].value_counts()
    
    print("\n✅ Analysis complete!")
    print(f"\n📊 Sentiment Summary:")
    print(f"   Positive: {sentiment_counts.get('positive', 0)}")
    print(f"   Negative: {sentiment_counts.get('negative', 0)}")
    print(f"   Neutral: {sentiment_counts.get('neutral', 0)}")
    
    return news_df

# Test it!
apple_news = get_news_with_sentiment("Apple", days_back=3)

# Show top 5 results
print("\n📈 Sample Results:")
print(apple_news[['title', 'sentiment', 'confidence']].head())

📰 Fetching news for Apple...
✅ Fetched 93 articles about Apple
🤖 Analyzing sentiment for 93 articles...

✅ Analysis complete!

📊 Sentiment Summary:
   Positive: 20
   Negative: 8
   Neutral: 65

📈 Sample Results:
                                               title sentiment  confidence
0                              mlx-guided-grpo 2.1.1   neutral    0.894080
1                      mlx-guided-grpo added to PyPI   neutral    0.857701
2  Bad Bunny to discuss Super Bowl halftime perfo...   neutral    0.917723
3  Get $600 Off This Lenovo ThinkBook With 16-Inc...   neutral    0.759157
4  Kotak Mahindra Bank to add 500 engineers amid ...  positive    0.822967


In [8]:
!pip install yfinance

In [9]:
import yfinance as yf
import numpy as np

def get_stock_data(ticker, days=30):
    """
    Fetch stock price data and calculate technical indicators
    """
    print(f"📊 Fetching stock data for {ticker}...")
    
    # Download data
    stock = yf.Ticker(ticker)
    df = stock.history(period=f"{days}d")
    
    if df.empty:
        print("❌ No stock data found!")
        return None
    
    # Calculate technical indicators
    df['SMA_5'] = df['Close'].rolling(window=5).mean()  # 5-day moving average
    df['SMA_20'] = df['Close'].rolling(window=20).mean()  # 20-day moving average
    df['Daily_Return'] = df['Close'].pct_change() * 100  # Daily % change
    
    print(f"✅ Got {len(df)} days of data")
    print(f"   Current Price: ${df['Close'][-1]:.2f}")
    print(f"   30-day Change: {((df['Close'][-1] / df['Close'][0] - 1) * 100):.2f}%")
    
    return df

# Test it!
apple_stock = get_stock_data("AAPL", days=30)
print("\n📈 Recent Prices:")
print(apple_stock[['Close', 'SMA_5', 'SMA_20', 'Daily_Return']].tail())

📊 Fetching stock data for AAPL...
✅ Got 30 days of data
   Current Price: $275.91
   30-day Change: 1.30%

📈 Recent Prices:
                                Close       SMA_5      SMA_20  Daily_Return
Date                                                                       
2026-01-30 00:00:00-05:00  259.480011  257.576001  257.649498      0.464617
2026-02-02 00:00:00-05:00  270.010010  260.496002  257.599498      4.058116
2026-02-03 00:00:00-05:00  269.480011  262.738007  257.710498     -0.196289
2026-02-04 00:00:00-05:00  276.489990  266.748004  258.416998      2.601298
2026-02-05 00:00:00-05:00  275.910004  270.274005  259.195999     -0.209768


C:\Users\User\AppData\Local\Temp\ipykernel_12700\3517585795.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"   Current Price: ${df['Close'][-1]:.2f}")
C:\Users\User\AppData\Local\Temp\ipykernel_12700\3517585795.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"   30-day Change: {((df['Close'][-1] / df['Close'][0] - 1) * 100):.2f}%")


In [10]:
import warnings
warnings.filterwarnings('ignore')

def generate_trading_signal(ticker, company_name, news_days=3, stock_days=30):
    """
    Generate trading signal based on sentiment + technical indicators
    """
    print(f"🎯 Generating trading signal for {ticker} ({company_name})")
    print("="*60)
    
    # 1. Get news sentiment
    news_df = get_news_with_sentiment(company_name, days_back=news_days)
    
    if news_df is None:
        return None
    
    # Calculate sentiment score (-1 to +1)
    sentiment_score = (
        news_df['sentiment'].value_counts().get('positive', 0) - 
        news_df['sentiment'].value_counts().get('negative', 0)
    ) / len(news_df)
    
    print(f"\n💭 Sentiment Score: {sentiment_score:.3f}")
    
    # 2. Get stock data
    stock_df = get_stock_data(ticker, days=stock_days)
    
    if stock_df is None:
        return None
    
    # 3. Technical indicators
    current_price = stock_df['Close'][-1]
    sma_5 = stock_df['SMA_5'][-1]
    sma_20 = stock_df['SMA_20'][-1]
    
    # Golden cross (bullish) vs Death cross (bearish)
    technical_signal = "BULLISH" if sma_5 > sma_20 else "BEARISH"
    
    print(f"📊 Technical Signal: {technical_signal}")
    print(f"   5-day MA: ${sma_5:.2f}")
    print(f"   20-day MA: ${sma_20:.2f}")
    
    # 4. Generate final signal
    print(f"\n{'='*60}")
    
    if sentiment_score > 0.1 and technical_signal == "BULLISH":
        signal = "🟢 STRONG BUY"
        reason = "Positive sentiment + bullish technicals"
    elif sentiment_score > 0.1:
        signal = "🟡 BUY"
        reason = "Positive sentiment, but mixed technicals"
    elif sentiment_score < -0.1 and technical_signal == "BEARISH":
        signal = "🔴 STRONG SELL"
        reason = "Negative sentiment + bearish technicals"
    elif sentiment_score < -0.1:
        signal = "🟠 SELL"
        reason = "Negative sentiment, but mixed technicals"
    else:
        signal = "⚪ HOLD"
        reason = "Neutral sentiment and technicals"
    
    print(f"\n🎯 SIGNAL: {signal}")
    print(f"📝 Reason: {reason}")
    print(f"{'='*60}\n")
    
    return {
        'ticker': ticker,
        'signal': signal,
        'sentiment_score': sentiment_score,
        'technical_signal': technical_signal,
        'current_price': current_price
    }

# Test it!
result = generate_trading_signal("AAPL", "Apple", news_days=3)

🎯 Generating trading signal for AAPL (Apple)
📰 Fetching news for Apple...
✅ Fetched 93 articles about Apple
🤖 Analyzing sentiment for 93 articles...

✅ Analysis complete!

📊 Sentiment Summary:
   Positive: 20
   Negative: 8
   Neutral: 65

💭 Sentiment Score: 0.129
📊 Fetching stock data for AAPL...
✅ Got 30 days of data
   Current Price: $275.91
   30-day Change: 1.30%
📊 Technical Signal: BULLISH
   5-day MA: $270.27
   20-day MA: $259.20


🎯 SIGNAL: 🟢 STRONG BUY
📝 Reason: Positive sentiment + bullish technicals



In [11]:
def backtest_strategy(ticker, company_name, start_date, end_date):
    """
    Backtest the trading strategy on historical data
    """
    print(f"📊 Backtesting {ticker} from {start_date} to {end_date}")
    print("="*60)
    
    # Get historical stock data
    stock = yf.Ticker(ticker)
    historical_data = stock.history(start=start_date, end=end_date)
    
    # Simple backtesting logic
    # We'll simulate: if sentiment > 0.1, BUY. If < -0.1, SELL. Else HOLD.
    
    initial_capital = 10000  # Start with $10,000
    shares = 0
    cash = initial_capital
    portfolio_value = []
    
    # For simplicity, we'll use a basic strategy:
    # Buy and hold vs our sentiment strategy
    
    buy_hold_shares = initial_capital / historical_data['Close'].iloc[0]
    buy_hold_value = buy_hold_shares * historical_data['Close'].iloc[-1]
    
    # Our strategy value (simplified - assumes we held based on initial signal)
    result = generate_trading_signal(ticker, company_name, news_days=7)
    
    if result and 'BUY' in result['signal']:
        our_shares = initial_capital / historical_data['Close'].iloc[-30]  # Bought 30 days ago
        our_value = our_shares * historical_data['Close'].iloc[-1]
    else:
        our_value = initial_capital  # Stayed in cash
    
    # Calculate returns
    buy_hold_return = ((buy_hold_value - initial_capital) / initial_capital) * 100
    our_return = ((our_value - initial_capital) / initial_capital) * 100
    
    print(f"\n💰 RESULTS:")
    print(f"   Initial Capital: ${initial_capital:,.2f}")
    print(f"\n   📈 Buy & Hold Strategy:")
    print(f"      Final Value: ${buy_hold_value:,.2f}")
    print(f"      Return: {buy_hold_return:.2f}%")
    print(f"\n   🤖 Our AI Strategy:")
    print(f"      Final Value: ${our_value:,.2f}")
    print(f"      Return: {our_return:.2f}%")
    print(f"\n   {'🎉 AI WINS!' if our_return > buy_hold_return else '📊 Buy & Hold Wins'}")
    print(f"   Difference: {abs(our_return - buy_hold_return):.2f}%")
    print("="*60)
    
    return {
        'buy_hold_return': buy_hold_return,
        'ai_return': our_return,
        'difference': our_return - buy_hold_return
    }

# Test it!
backtest_results = backtest_strategy("AAPL", "Apple", "2025-01-01", "2026-02-06")

📊 Backtesting AAPL from 2025-01-01 to 2026-02-06
🎯 Generating trading signal for AAPL (Apple)
📰 Fetching news for Apple...
✅ Fetched 93 articles about Apple
🤖 Analyzing sentiment for 93 articles...

✅ Analysis complete!

📊 Sentiment Summary:
   Positive: 18
   Negative: 10
   Neutral: 65

💭 Sentiment Score: 0.086
📊 Fetching stock data for AAPL...
✅ Got 30 days of data
   Current Price: $275.91
   30-day Change: 1.30%
📊 Technical Signal: BULLISH
   5-day MA: $270.27
   20-day MA: $259.20


🎯 SIGNAL: ⚪ HOLD
📝 Reason: Neutral sentiment and technicals


💰 RESULTS:
   Initial Capital: $10,000.00

   📈 Buy & Hold Strategy:
      Final Value: $11,365.92
      Return: 13.66%

   🤖 Our AI Strategy:
      Final Value: $10,000.00
      Return: 0.00%

   📊 Buy & Hold Wins
   Difference: 13.66%


In [12]:
!pip install streamlit plotly